<a href="https://colab.research.google.com/github/areias/viral-escape/blob/main/viral_escape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Learning the language of viral escape
1.   paper 
2.   supplementary material
3.   code https://github.com/brianhie/viral-mutation/blob/master/bin/language_model.py


> used a BiLSTM architecture with two hidden layers of 512
hidden units each, with an Adam learning rate of 0.001. We used the same architecture for all
experiments. We train the language model to predict the observed amino acid residue at all
positions in each sequence, using the remaining sequence as the input; one training epoch is
completed when the model has considered all positions in all sequences in the training corpus.
We trained each model until convergence of cross entropy loss across one training epoch.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jan  3 21:38:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


Training a new model on flu HA sequences can be done with the command


In [27]:
#! python bin/flu.py bilstm --train --test > flu_train.log 2>&1

In [4]:
# clone repository
! git clone https://github.com/brianhie/viral-mutation.git

fatal: destination path 'viral-mutation' already exists and is not an empty directory.


In [4]:
# download data
!wget http://cb.csail.mit.edu/cb/viral-mutation/data.tar.gz


--2022-01-03 18:31:37--  http://cb.csail.mit.edu/cb/viral-mutation/data.tar.gz
Resolving cb.csail.mit.edu (cb.csail.mit.edu)... 128.30.2.148
Connecting to cb.csail.mit.edu (cb.csail.mit.edu)|128.30.2.148|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3060938794 (2.9G) [application/x-gzip]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>]   2.85G  40.8MB/s    in 73s     

2022-01-03 18:32:51 (40.0 MB/s) - ‘data.tar.gz’ saved [3060938794/3060938794]



In [5]:
# unzip data
!tar xvf data.tar.gz

data/
data/escape_results.txt
data/evcouplings/
data/evcouplings/flu_h3_config.yaml
data/evcouplings/flu_h1_config.yaml
data/evcouplings/hiv_bf520_seq.fa
data/evcouplings/hiv_env_seq.fa
data/evcouplings/hiv_env_config.yaml
data/evcouplings/sarscov2_config.yaml
data/evcouplings/flu_h1_seq.fa
data/evcouplings/hiv_bf520_config.yaml
data/evcouplings/sarscov2_seq.fa
data/evcouplings/flu_h3_seq.fa
data/headlines/
data/headlines/abcnews-date-text.csv
data/headlines/headlines.txt
data/hiv/
data/hiv/bg505_regions.txt
data/hiv/fitness_haddox2018/
data/hiv/fitness_haddox2018/BG505-1_prefs.csv
data/hiv/fitness_haddox2018/BG505-3_prefs.csv
data/hiv/fitness_haddox2018/BG505-2_prefs.csv
data/hiv/fitness_haddox2018/BF520-3_prefs.csv
data/hiv/fitness_haddox2018/BF520-2_prefs.csv
data/hiv/fitness_haddox2018/BF520-1_prefs.csv
data/hiv/fitness_haddox2018/BF520_to_HXB2.csv
data/hiv/fitness_haddox2018/BF520_env.fasta
data/hiv/fitness_haddox2018/map_indices.py
data/hiv/fitness_haddox2018/BF520_avgprefs.csv
d

In [7]:
# install dependencies
! pip install anndata 

     |████████████████████████████████| 91 kB 6.9 MB/s 


In [8]:
! pip install scanpy

     |████████████████████████████████| 2.0 MB 9.7 MB/s 
     |████████████████████████████████| 86 kB 7.4 MB/s 
     |████████████████████████████████| 1.1 MB 67.6 MB/s 
     |████████████████████████████████| 63 kB 2.7 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82709 sha256=9d46bb49c5206cc79de82efd96b4fa2487a4cc975df143f8cf4a79a675678859
  Stored in directory: /root/.cache/pip/wheels/84/1b/c6/aaf68a748122632967cef4dffef68224eb16798b6793257d82
  Created wheel for pynndescent: filename=pynndescent-0.5.5-py3-none-any.whl size=52603 sha256=108d041a692e58114f4f0d3845b2b2c27460f5da67d0c39f91701ce1d05681a1
  Stored in directory: /root/.cache/pip/wheels/af/e9/33/04db1436df0757c42fda8ea6796d7a8586e23c85fac355f476
  Created wheel for sinfo: filename=sinfo-0.3.4-py3-none-any.whl size=7899 sha256=8f5d0b39ae384fc94e3ec9b446d32e6d7852d106f172b32b27cb832587f85c2e
  Stored in directory: /root/.cache/pip/wheels/68/ca/56/344d532fe53e855ccd6549795d370588ab8123

In [9]:
! pip install bio


     |████████████████████████████████| 271 kB 8.4 MB/s 
     |████████████████████████████████| 2.3 MB 78.7 MB/s 


In [3]:
# add mutation to path
import sys
sys.path.append('viral-mutation/bin')

In [4]:
# from bin/flu.py
from mutation import *



In [5]:
## creating arguments dict

"""
def parse_args():
    import argparse
    parser = argparse.ArgumentParser(description='Flu sequence analysis')
    parser.add_argument('model_name', type=str,
                        help='Type of language model (e.g., hmm, lstm)')
    parser.add_argument('--namespace', type=str, default='flu',
                        help='Model namespace')
    parser.add_argument('--dim', type=int, default=512,
                        help='Embedding dimension')
    parser.add_argument('--batch-size', type=int, default=1000,
                        help='Training minibatch size')
    parser.add_argument('--n-epochs', type=int, default=14,
                        help='Number of training epochs')
    parser.add_argument('--seed', type=int, default=1,
                        help='Random seed')
    parser.add_argument('--checkpoint', type=str, default=None,
                        help='Model checkpoint')
    parser.add_argument('--train', action='store_true',
                        help='Train model')
    parser.add_argument('--train-split', action='store_true',
                        help='Train model on portion of data')
    parser.add_argument('--test', action='store_true',
                        help='Test model')
    parser.add_argument('--embed', action='store_true',
                        help='Analyze embeddings')
    parser.add_argument('--semantics', action='store_true',
                        help='Analyze mutational semantic change')
    parser.add_argument('--combfit', action='store_true',
                        help='Analyze combinatorial fitness')
    args = parser.parse_args()
    return args
"""
from collections import namedtuple
arguments = namedtuple('arguments', ['model_name','train','test',
                                     'dim','n_epochs','batch_size',
                                     'namespace', 'seed','checkpoint','train_split'])

args = arguments('bilstm', True, True,
                 512,3,350, # defaults were batch-size 1000, 14 epochs
                 'flu',1, None,None)
args

arguments(model_name='bilstm', train=True, test=True, dim=512, n_epochs=3, batch_size=350, namespace='flu', seed=1, checkpoint=None, train_split=None)

In [6]:
AAs = [
        'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H',
        'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W',
        'Y', 'V', 'X', 'Z', 'J', 'U', 'B', 'Z'
    ]
    
vocabulary = { aa: idx + 1 for idx, aa in enumerate(sorted(AAs)) }



In [7]:
vocabulary

{'A': 1,
 'B': 2,
 'C': 3,
 'D': 4,
 'E': 5,
 'F': 6,
 'G': 7,
 'H': 8,
 'I': 9,
 'J': 10,
 'K': 11,
 'L': 12,
 'M': 13,
 'N': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 26}

In [8]:
def setup(args):
    fnames = [ 'data/influenza/ird_influenzaA_HA_allspecies.fa' ]
    meta_fnames = [ 'data/influenza/ird_influenzaA_HA_allspecies_meta.tsv' ]

    seqs = process(fnames, meta_fnames)

    seq_len = max([ len(seq) for seq in seqs ]) + 2
    vocab_size = len(AAs) + 2

    model = get_model(args, seq_len, vocab_size)

    return model, seqs

In [9]:
def process(fnames, meta_fnames):
    metas = load_meta(meta_fnames)

    seqs = {}
    for fname in fnames:
        for record in SeqIO.parse(fname, 'fasta'):
            if 'Reference_Perth2009_HA_coding_sequence' in record.description:
                continue
            if str(record.seq).count('X') > 10:
                continue
            if record.seq not in seqs:
                seqs[record.seq] = []
            accession = record.description.split('|')[0].split(':')[1]
            seqs[record.seq].append(metas[accession])
    return seqs

In [12]:
def load_meta(meta_fnames):
    metas = {}
    for fname in meta_fnames:
        with open(fname) as f:
            header = f.readline().rstrip().split('\t')
            for line in f:
                fields = line.rstrip().split('\t')
                accession = fields[1]
                meta = {}
                for key, value in zip(header, fields):
                    if key == 'Subtype':
                        meta[key] = value.strip('()').split('N')[0].split('/')[-1]
                    elif key == 'Collection Date':
                        meta[key] = int(value.split('/')[-1]) \
                                    if value != '-N/A-' else None
                    elif key == 'Host Species':
                        meta[key] = value.split(':')[1].split('/')[-1].lower()
                    else:
                        meta[key] = value
                metas[accession] = meta
    return metas

In [13]:
model, seqs = setup(args)

In [14]:
model

In [15]:
len(seqs)

44851

In [16]:
#seqs

In [17]:
from collections import defaultdict 

seqs_subset= defaultdict(dict)

for x in list(seqs)[0:1000]:
    seqs_subset[x] = seqs[x]


In [18]:
len(seqs_subset)

1000

In [19]:
def split_seqs(seqs, split_method='random'):
    train_seqs, test_seqs, val_seqs = {}, {}, {}

    old_cutoff = 1990
    new_cutoff = 2018

    tprint('Splitting seqs...')
    for seq in seqs:
        # Pick validation set based on date.
        seq_dates = [
            meta['Collection Date'] for meta in seqs[seq]
            if meta['Collection Date'] is not None
        ]
        if len(seq_dates) > 0:
            oldest_date = sorted(seq_dates)[0]
            if oldest_date < old_cutoff or oldest_date >= new_cutoff:
                test_seqs[seq] = seqs[seq]
                continue
        train_seqs[seq] = seqs[seq]
    tprint('{} train seqs, {} test seqs.'
           .format(len(train_seqs), len(test_seqs)))

    return train_seqs, test_seqs



In [20]:
if args.train or args.train_split or args.test:
        train_test(args, model, seqs_subset, vocabulary, split_seqs)


Epoch 1/3
1621/1621 [==============================] - 2294s 1s/step - loss: 2.9451 - accuracy: 0.0779
Epoch 2/3
1621/1621 [==============================] - 2291s 1s/step - loss: 1.2038 - accuracy: 0.6537
Epoch 3/3
1621/1621 [==============================] - 2288s 1s/step - loss: 0.1340 - accuracy: 0.9692


In [26]:
train_seqs, test_seqs = split_seqs(seqs_subset)

2022-01-03 20:58:53.092750 | Splitting seqs...
2022-01-03 20:58:53.120300 | 824 train seqs, 176 test seqs.


In [ ]:
# saved model in target/flu/checkpoints/bilstm > saved to vsp project locally


Influenza HA semantic embedding UMAPs and log files with statistics can be generated with the command

In [11]:
! python viral-mutation/bin/flu.py bilstm --checkpoint target/flu/checkpoints/bilstm/bilstm_512-03.hdf5 --embed > flu_embed.log 2>&1

/bin/bash: line 1:  2880 Killed                  python viral-mutation/bin/flu.py bilstm --checkpoint target/flu/checkpoints/bilstm/bilstm_512-03.hdf5 --embed > flu_embed.log 2>&1


Single-residue escape prediction using validation data from Doud et al. (2018) and Lee et al. (2019) can be done with the command

In [7]:
! python viral-mutation/bin/flu.py bilstm --checkpoint target/flu/checkpoints/bilstm/bilstm_512-03.hdf5 --semantics \
    > flu_semantics.log 2>&1

/bin/bash: line 1:  2735 Killed                  python viral-mutation/bin/flu.py bilstm --checkpoint target/flu/checkpoints/bilstm/bilstm_512-03.hdf5 --semantics > flu_semantics.log 2>&1


Combinatorial fitness experiments measuring correlation with grammaticality and semantic change using data from Doud and Bloom (2016) and from Wu et al. (2020) can be done with the command

In [10]:
! python viral-mutation/bin/flu.py bilstm --checkpoint target/flu/checkpoints/bilstm/bilstm_512-03.hdf5 --combfit \
    > flu_combfit.log 2>&1

/bin/bash: line 1:  2824 Killed                  python viral-mutation/bin/flu.py bilstm --checkpoint target/flu/checkpoints/bilstm/bilstm_512-03.hdf5 --combfit > flu_combfit.log 2>&1


## 1. The Data

We obtained mutational fitness preference scores for HA H1 A/WSN/1933 (WSN33) mutants from [Doud and Bloom (17) ](https://www.mdpi.com/1999-4915/8/6/155)


M. B. Doud, J. D. Bloom, Accurate measurement of the effects of all amino-acid mutations
on influenza hemagglutinin. Viruses 8, 155 (2016). doi:10.3390/v8060155 Medline 
> enabling us to measure with greatly improved accuracy and reproducibility the effects of all amino-acid mutations to an H1 influenza hemagglutinin on viral replication in cell culture.

> Our measurements confirm at much higher resolution the results of previous studies suggesting that antigenic sites on the globular head of hemagglutinin are highly tolerant of mutations. We also show that other regions of hemagglutinin—including the stalk epitopes targeted by broadly neutralizing antibodies—have a much lower inherent capacity to tolerate point mutations. The ability to accurately measure the effects of all influenza mutations should enhance efforts to understand and predict viral evolution







In [ ]:
import pandas as pd


In [ ]:
df_e = pd.read_csv("data/target/escape_results.txt", )

SyntaxError: ignored

## Model predictions

> For the CSCS, our empirical results seem consistently well-calibrated around β = 1 (equally weighting both terms), which we used in all of our experiments.

> We use different aspects of the language model to describe semantic change and grammaticality. Semantic change is defined as the L1 norm of the between the average embedding across all positions of the original sequence and that of the mutated sequence, . The l 1 norm, chosen because of more favorable properties compared to other standard distance metrics Effectively, distances in embedding space approximate semantic change and the emitted probability approximates grammaticality.



viral-mutation/results/flu/semantics/






In [9]:
## Escape prediction validation

> We obtained experimentally validated causal escape mutations to HA H1 WSN33 from
Doud et al. (1), HA H3 Perth09 from Lee et al. (2)
M. B. Doud, J. M. Lee, J. D. Bloom, How single mutations affect viral escape from broad and
narrow antibodies to H1 influenza hemagglutinin. Nat. Commun. 9, 1386 (2018).
doi:10.1038/s41467-018-03665-3 Medline
J. M. Lee, R. Eguia, S. J. Zost, S. Choudhary, P. C. Wilson, T. Bedford, T. Stevens-Ayers, M.
Boeckh, A. C. Hurt, S. S. Lakdawala, S. E. Hensley, J. D. Bloom, Mapping person-to-
person variation in viral mutations that escape polyclonal serum targeting influenza
hemagglutinin. eLife 8, e49324 (2019). doi:10.7554/eLife.49324 Medline
We then made, in silico, all possible single-residue mutations to H1 WSN33, H3 Perth09…. for each of these mutations, we computed semantic change and grammaticality and combined these scores using the CSCS rank-based acquisition function as described in Note S2. For a given viral protein, the value of the CSCS acquisition function was used to rank all possible mutants.
 




Here, we map all single amino-acid mutations that increase resistance to broad antibodies to H1 hemagglutinin. Our approach not only identifies antigenic mutations but also quantifies their effect sizes. All antibodies select mutations, but the effect sizes vary widely. The virus can escape a broad antibody to hemagglutinin’s receptor-binding site the same way it escapes narrow strain-specific antibodies: via single mutations with huge effects. In contrast, broad antibodies to hemagglutinin’s stalk only select mutations with small effects. Therefore, among the antibodies we examine, breadth is an imperfect indicator of the potential for viral escape via single mutations. Antibodies targeting the H1 hemagglutinin stalk are quantifiably harder to escape than the other antibodies tested here.

SyntaxError: ignored

## Escape prediction validation

> We obtained experimentally validated causal escape mutations to HA H1 WSN33 from
Doud et al. (1), HA H3 Perth09 from Lee et al. (2)

> We then made, in silico, all possible single-residue mutations to H1 WSN33, H3 Perth09…. for each of these mutations, we computed semantic change and grammaticality and combined these scores using the CSCS rank-based acquisition function as described in Note S2. For a given viral protein, the value of the CSCS acquisition function was used to rank all possible mutants.
 




M. B. Doud, J. M. Lee, J. D. Bloom,[How single mutations affect viral escape from broad and
narrow antibodies to H1 influenza hemagglutinin.](https://www.nature.com/articles/s41467-018-03665-3.pdf) Nat. Commun. 9, 1386 (2018).
doi:10.1038/s41467-018-03665-3 Medline

> Here, we map all single amino-acid mutations that increase resistance to broad antibodies to H1 hemagglutinin. Our approach not only identifies antigenic mutations but also quantifies their effect sizes. All antibodies select mutations, but the effect sizes vary widely. The virus can escape a broad antibody to hemagglutinin’s receptor-binding site the same way it escapes narrow strain-specific antibodies: via single mutations with huge effects. In contrast, broad antibodies to hemagglutinin’s stalk only select mutations with small effects. Therefore, among the antibodies we examine, breadth is an imperfect indicator of the potential for viral escape via single mutations. Antibodies targeting the H1 hemagglutinin stalk are quantifiably harder to escape than the other antibodies tested here.


J. M. Lee, R. Eguia, S. J. Zost, S. Choudhary, P. C. Wilson, T. Bedford, T. Stevens-Ayers, M.
Boeckh, A. C. Hurt, S. S. Lakdawala, S. E. Hensley, J. D. Bloom, [Mapping person-to-
person variation in viral mutations that escape polyclonal serum targeting influenza
hemagglutinin.](https://elifesciences.org/articles/49324.pdf) eLife 8, e49324 (2019). doi:10.7554/eLife.49324 Medline

> Here, we map how all amino-acidmutations to influenza’s major surface protein affect viral neutralization by polyclonal human sera.